In [1]:
import sys
# add the path
sys.path.append("../../")
from base_utils import *
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
from functools import partial
from transformers import AutoTokenizer, AutoModel, BartForConditionalGeneration, BertForSequenceClassification
import argparse
from nltk.tokenize import sent_tokenize
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
class Processor():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        
    def __call__(self, sample):
        for key in sample.keys():
            if key == "input": 
                text = self.tokenizer(sample[key])["input_ids"]
            if key == "target":
                if sample[key]: #True
                    target = 1  
                else:
                    target = 0
        return_dict = {"input_ids": torch.tensor(text), 
                        "target": torch.tensor([target])}
        return return_dict
    
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
path = "./train_classifier_data/"
train = os.path.join(path, "new_train.csv")
valid = os.path.join(path, "new_valid.csv")
test = os.path.join(path, "new_test.csv")
data = load_dataset("csv", data_files={"train": train,
                                       "valid": valid,
                                       "test": test})
batch_size = 8
processor = Processor(tokenizer)
encoded_data = data.map(lambda sample: processor(sample))
encoded_data.set_format("torch")
# batchify the encoded data
train_dataloader = batchify(encoded_data["train"]["input_ids"], encoded_data["train"]["target"],
                            batch_size=batch_size)
valid_dataloader = batchify(encoded_data["valid"]["input_ids"], encoded_data["valid"]["target"],
                            batch_size=batch_size)
test_dataloader = batchify(encoded_data["test"]["input_ids"], encoded_data["test"]["target"],
                           batch_size=batch_size)

Using custom data configuration default-56aad127a647ced7


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

/home/yu/anaconda3/envs/venv/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


0 tables [00:00, ? tables/s]

...

/home/yu/anaconda3/envs/venv/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?ex/s]

  0%|          | 0/200 [00:00<?, ?ex/s]

  0%|          | 0/200 [00:00<?, ?ex/s]

In [3]:
class Classifier(JoModule):
    def __init__(self, base):
        super().__init__()
        self.base = base
        self.loss_fn = nn.CrossEntropyLoss()
        
    def forward(self, input_ids):
        logits = self.base(input_ids).logits
        return logits   # in shape (N, 2)
    
    def training_step(self, batch, device):
        input_ids, target = batch
        input_ids = input_ids.to(device)
        target = target.reshape(-1).to(device)
        logits = self.forward(input_ids)
        loss = self.loss_fn(logits, target)
        return loss
        
    
    def validation_step(self, batch, device, metrics=["loss", "precision"]):
        input_ids, target = batch
        input_ids = input_ids.to(device)
        target = target.reshape(-1).to(device)
        logits = self.forward(input_ids)
        # compute loss
        loss = self.loss_fn(logits, target)
        # compute precision
        precision = (logits.topk(1).indices.reshape(-1) == target).sum() / len(target)
        return {"loss": loss, "precision": precision}
    
base = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model = Classifier(base)
trainer = Trainer(batch_size=8,
                  max_epochs=5,
                  optimizer_method="Adam",
                  lr=2e-6,
                  save_model="exp_classifier_new",
                  logging="exp_classifier.log",
                  use_amp=False,
                  warmup=False,
                  accelerator="cuda:1",
                  valid_metrics=["loss", "precision"],
                  efficient_valid=True, 
                  )
trainer.fit(model, train_dataloader, valid_dataloader)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Start training...
The total number of parameters is: 109.48M
current learning rate: 2e-06


  0%|          | 0/213 [00:00<?, ?it/s]

  0%|          | 0/213 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

...

------------------------- epoch: 3 -------------------------
on train data
loss: 0.32770800590515137
precision: 0.9103773832321167
------------------------- epoch: 3 -------------------------
on valid data
loss: 0.400614470243454
precision: 0.8537323474884033
current learning rate: 2e-06


  0%|          | 0/213 [00:00<?, ?it/s]

  0%|          | 0/213 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

------------------------- epoch: 4 -------------------------
on train data
loss: 0.272190660238266
precision: 0.9386792778968811
------------------------- epoch: 4 -------------------------
on valid data
loss: 0.3869752585887909
precision: 0.8575933575630188


In [4]:
model = Classifier(base)
model = load_model(model, "./saved/exp_classifier_new_dict")
model.validIter(test_dataloader, "cuda:1", ["loss", "precision"], False)

  0%|          | 0/37 [00:00<?, ?it/s]

defaultdict(float,
            {'loss': tensor(0.4331, device='cuda:1'),
             'precision': tensor(0.8378, device='cuda:1')})